In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import normalize

In [240]:
#####take diabetes dataset
data=pd.read_csv("C:/Users/HP/Documents/Tution Classes/Logistic_Regression/diabetes.csv")

In [241]:
data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [242]:
data.describe()


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [243]:
###### first break into train and val 
def train_val_data(n,n_val):
    val_len=int(n_val*n)
    train_len=n-val_len
    idx=np.random.permutation(n)
    return idx[:train_len], idx[train_len:]

In [244]:
train_idx,val_idx=train_val_data(len(data),0.2)

In [245]:
X_train=data.iloc[train_idx].values
X_train=X_train[:,:8]
Y_train=data.iloc[train_idx].values
Y_train=Y_train[:,8]
X_val=data.iloc[val_idx].values
X_val=X_val[:,:8]
Y_val=data.iloc[val_idx].values
Y_val=Y_val[:,8]

In [246]:
#####normalize the dataset
X_train=normalize(X_train,norm='l1')
X_val=normalize(X_val,norm='l1')

In [11]:
##### create MLP

In [291]:
class MLP:
    def __init__(self,X_train,Y_train,X_val,Y_val):
        self.X_train=X_train
        self.Y_train=Y_train
        self.X_val=X_val
        self.Y_val=Y_val
        self.input_size=X_train.shape[1]
        self.h_size=5
        self.output_size=1
        self.W1,self.b1,self.W2,self.b2=self.init_weights()
#         print("Init ",self.W1)
        
    def init_weights(self):
        np.random.seed(7)
        self.W1=np.random.randn(self.input_size,self.h_size)
        self.b1=np.random.randn(1,self.h_size)
        self.W2=np.random.randn(self.h_size,self.output_size)
        self.b2=np.random.randn(1,self.output_size)
        return self.W1,self.b1,self.W2,self.b2
        
    def forward(self,x):
#         print("Foward :",self.W1)
        z1=x.dot(self.W1)+self.b1
        a1=1/(1 + np.exp(-z1))
        z2=a1.dot(self.W2)+self.b2
        a2=1/(1 + np.exp(-z2))
#         y_out=(a2>0.9).astype(int)  
#         print(a2.shape, y_out.shape)
#         print(a2)
        return a1,a2,a2
    
    def backward(self,y_pred,y_act,a1,a2,learning_rate):
        y_act = y_act.reshape((y_pred.shape[0],1))
#         print(delta2.shape)
        delta2=(a2.dot((1-a2).T)).dot(a2-y_act)
        print("Delta 2 : ", ((1-a2)))
#         print("delta2 shape:",delta2.shape,"a1 shape:",a1.shape,"W2 shape:",self.W2.shape)
        self.W2=self.W2-learning_rate*((delta2.T.dot(a1)).T)
        self.b2=self.b2-learning_rate*delta2
        delta1=self.W2.dot((((a1.dot((1-a1).T)).dot(delta2)).T))
        self.W1=self.W1-learning_rate*((delta1.dot(self.X_train)).T)
#         print("Delta 1 : ",delta1)
#         print("backprop :",self.W1)
        self.b1=self.b1-learning_rate*delta1.T
        
    def loss(self,y_pred,y_act):
        l=0.0
        for i in range(y_pred.shape[0]):
            l+=(y_pred[i]-y_act[i])**2
        return l
    
    def train(self,epoch,learning_rate):
        training_loss=[]
        for e in range(epoch):
            a1,a2,y_pred=self.forward(self.X_train)
#             print("a1:",a1)
#             print("a2:",a2)
            ls=self.loss(y_pred,self.Y_train)
            training_loss.append(ls)
            self.backward(y_pred,self.Y_train,a1,a2,learning_rate)
#             print(self.W1)
        return training_loss
            
        
        
    
        
        
        
        
        

In [292]:
obj=MLP(X_train,Y_train,X_val,Y_val)
# z1.shape
obj.train(4,0.01)

Delta 2 :  [[0.11779357]
 [0.12280475]
 [0.09994159]
 [0.11361496]
 [0.11405606]
 [0.13217674]
 [0.06194566]
 [0.12072209]
 [0.12740894]
 [0.10804077]
 [0.08146345]
 [0.10846039]
 [0.12928907]
 [0.14205069]
 [0.11423585]
 [0.11642523]
 [0.12692438]
 [0.1231064 ]
 [0.10785833]
 [0.12960577]
 [0.12429265]
 [0.11988718]
 [0.14384596]
 [0.12829512]
 [0.13311934]
 [0.11379323]
 [0.12687869]
 [0.12600847]
 [0.1172595 ]
 [0.1506817 ]
 [0.11336145]
 [0.14174763]
 [0.11000151]
 [0.11580847]
 [0.11968877]
 [0.14596204]
 [0.11530293]
 [0.11000334]
 [0.10462028]
 [0.12528529]
 [0.11044135]
 [0.13390819]
 [0.11519499]
 [0.12057569]
 [0.12402028]
 [0.12263423]
 [0.12594933]
 [0.1235666 ]
 [0.11032578]
 [0.11713727]
 [0.12445037]
 [0.12485686]
 [0.11508794]
 [0.11332101]
 [0.11480218]
 [0.10530722]
 [0.13465134]
 [0.14351901]
 [0.13695705]
 [0.11665886]
 [0.13889792]
 [0.12610112]
 [0.13127994]
 [0.11886936]
 [0.12941719]
 [0.09685144]
 [0.10950959]
 [0.09741799]
 [0.120475  ]
 [0.13164108]
 [0.12558

[array([311.60038299]), array([220.]), array([220.]), array([220.])]